In [ ]:
# -*- coding: UTF-8 -*-

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
#from google.colab import drive
#import zipfile
#import seaborn as sns
#import spacy as sp
import string
import random
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from textblob import TextBlob

[nltk_data] Downloading package punkt to /home/pcorrea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#.mount("/content/drive")

In [7]:
cols = ['title','reviewId', 'restaurantId','author','reviewBody','language','mobileDevice','userLocation','publicationDate','visitDate','ratingValue','itemReviewed','address','addressLocality','addressRegion','postalCode','country','website','email','phoneNumber','averageRating']
train_data = pd.read_csv('pc-dataset-v2.dat', names= cols, engine = 'python', delimiter=',', skiprows=0, dtype="string")
train_data.describe()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating
count,100000,100000,99999,99999,99999,99983,100000,100000,100000,100000,...,100000,99938,86319,100000,95036,100000,100000,100000,100000,100000
unique,59592,100000,14981,67202,99996,9,2,4114,3106,118,...,13579,14724,23,27,8029,1,7169,3110,9868,9
top,Muito bom,261874950,1098021,None,Indico para o melhores amigos! \u00E9 sem duvi...,pt,False,São Paulo. SP,30/07/2013,None,...,Coco Bambu Frutos do Mar,Av. Beira Mar. 3698 Bairro Mucuripe,S\u00E3o Paulo,Estado de S\u00E3o Paulo,59090-000,Brasil,None,None,None,4.5
freq,1803,1,1083,813,2,99013,57599,14707,200,4330,...,1083,1083,17455,17455,1418,100000,14473,51782,7655,48989


In [6]:
train_data.loc[pd.isnull(train_data['phoneNumber'])]

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating


In [8]:
train_data.head()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating
0,Jantar,692856288,2344932,larinhavercosa222,Comida maravilhosa. lugar do verdadeiro chicle...,pt,True,São Carlos. SP,26/07/2019,07/2019,...,Imperador dos Camar\u00F5es,Avenida Doutor Ant\u00F4nio Gouveia,<NA>,Alagoas,57030-170,Brasil,http://www.imperadordoscamaroes.com.br,contato@imperadordoscamaroes.com.br,+55 82 3231-4134,4.5
1,Uma boa experi\u00EAncia.,229208487,3720906,Pano P,Comida boa. drinks legais e um ambiente bem ac...,pt,True,Manaus. AM,16/09/2014,None,...,Guacamole Natal,Avenida Praia de Ponta Negra 9068 Ponta Negra,<NA>,Rio Grande do Norte,59092-100,Brasil,http://www.facebook.com/GuacamoleNatal,guacamolenatal@gmail.com,+55 84 3219-3646,4.5
2,Otimo self service,679002332,2248105,Magma100,Excelente pra ir com a fam\u00EDlia! \u00D3tim...,pt,True,Goiânia. GO,04/06/2019,07/2018,...,Chica Pitanga,Rua Petrolina 19,Recife,Pernambuco,51021-250,Brasil,http://www.chicapitanga.com.br,chicapitanga@chicapitanga.com.br,+55 81 3334-8260,4.5
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,Aracaju. SE,15/03/2016,01/2016,...,Sorvetes Vi Sabor,Avenida Francisco Porto 59 Grageru,Aracaju,Sergipe,<NA>,Brasil,None,None,None,4.0
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que j\u00E1 comi! Excelente...,pt,False,Rio de Janeiro. RJ,28/04/2015,04/2015,...,Z Deli Sandwich Shop,Rua Haddock Lobo 1386 Jardins,S\u00E3o Paulo,Estado de S\u00E3o Paulo,01414-002,Brasil,http://www.facebook.com/zdelisandwichshop/,None,551130830021,4.5


In [9]:
train_data.tail()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,...,itemReviewed,address,addressLocality,addressRegion,postalCode,country,website,email,phoneNumber,averageRating
99995,Nova op\u00E7\u00E3o no Paseo!!!,672210394,17424548,Fabiano B,Ainda n\u00E3o inaugurado oficialmente. est\u0...,pt,True,Porto Alegre. RS,07/05/2019,05/2019,...,Boteco Maestro,Avenida Wenceslau Escobar 1823,Porto Alegre,Rio Grande do Sul,91900-000,Brasil,http://www.facebook.com/botecomaestro,None,+55 51 99353-6602,4.0
99996,Bom,380904400,2440184,Chris L,Bom atendimento. lugar bonito. comida muito go...,pt,True,Cork. Irlanda,08/06/2016,01/2016,...,Casa do Marqu\u00EAs,Rua Marques do Pombal. 1814 Higien\u00F3polis,Porto Alegre,Rio Grande do Sul,90540-000,Brasil,http://galeteriacasadomarques.com.br/,galeteriacasadomarques@galeteriacasadomarques....,+55 51 3343-4303,4.5
99997,Bom bar,186111858,2392996,Guilherme P,O Rio Scenarium \u00E9 uma das baladas mais fa...,pt,False,São Paulo. SP,27/11/2013,11/2013,...,Rio Scenarium,Rua do Lavradio 20 Centro,Rio de Janeiro,Estado do Rio de Janeiro,20230-070,Brasil,http://www.rioscenarium.art.br,None,+55 21-3147-9000,4.5
99998,Me senti enganada...,152745909,1062290,LisianeCohem,Fui por indica\u00E7\u00E3o e com boa expectat...,pt,True,Porto Alegre. RS,21/02/2013,None,...,Celeiro,Rua Dias Ferreira 199,Rio de Janeiro,Estado do Rio de Janeiro,22431-050,Brasil,http://www.celeiroculinaria.com.br,celeiro@celeiroculinaria.com.br,55 21 2274 7843,3.5
99999,Restaurante excelente,649755498,12433911,brunotulio,Experi\u00EAncia fant\u00E1stica com comida de...,pt,True,Manaus. AM,02/02/2019,02/2019,...,Coco Bambu Manaus,Avenida Coronel Teixeira. 5705 Ponta Negra. Sh...,Manaus,Rio Amazonas,69037-000,Brasil,http://www.facebook.com/cocobambumanausoficial/,rh.manaus@cocobambu.com,+55 92 3667-7430,5.0


In [ ]:
_train_data = train_data.reviewBody.astype("string")
type(_train_data)
_train_data

In [ ]:
# _blob = TextBlob(_train_data))
# print(_blob.translate(to='en'))

In [ ]:
data = train_data
X = data.iloc[:, 4].values # aqui pode ser selecionado a coluna

In [ ]:
_X = X
print(_X)

In [ ]:
# test = np.array_str(X).encode('utf-16', 'surrogatepass').decode('utf-16')
# serieTest = pd.Series(test)
# print(serieTest)

In [ ]:
serie =  pd.Series(np.array_str(X))
serie

In [ ]:
blob = TextBlob(serie[0])
print(blob.translate(to='en'))

In [ ]:
dic2 = pd.DataFrame(train_data)
dic2['reviewBody']
count = 0
for x in dic2['reviewBody']:
    count = count + 1  
    print(type(x))
    print(x) #vai imprimir todos os reviews
    blob = TextBlob(x)
    print(blob.translate(to='en'))
    if count == 5:
        break

In [ ]:
result = []
x = []
for _data in data:
  print(type(_data), _data)
  if _data == 'reviewBody':    
    for x in _data:
      print()
    print(result)

In [ ]:
for x in serie:
  print(x)
  blob = TextBlob(x)
  print(blob.translate(to='en'))

In [ ]:
train_data

In [ ]:
def limparDados(text):
    text = re.sub()
    return text

In [ ]:
X

In [ ]:
_X = np.array_str(X)
print(_X)
#_X.decode('unicode-escape')
X.shape #Formato 

In [ ]:
type(X) # tipo do dado

In [ ]:
data_str = _X[0:10000]
str(data_str).encode('unicode_escape').decode('unicode_escape')

In [ ]:
def decodeUnicodeArray(_string):
    espacoSeq = _string.group(0)
    digito = espacoSeq[2:]
    ordinal = int(digito, 16)
    char = chr(ordinal)
    return char
    

In [ ]:
re.sub(r'(\\u[09A-Fa-f]+)', decodeUnicodeArray, "test")

In [ ]:
train_data.describe()

In [ ]:
#data.drop(['title', 'restaurantId','author','reviewBody','language','userLocation','publicationDate','visitDate','ratingValue','itemReviewed','address','addressLocality','addressRegion','postalCode','country','website','email','phoneNumber','averageRating'], axis = 1, inplace=True)

In [ ]:
#train_data.mobileDevice.unique()

In [ ]:
sns.countplot(train_data.mobileDevice);

In [ ]:
sns.catplot(x="mobileDevice", kind="count", palette="ch:.25", data=train_data)

In [ ]:
%pwd

In [ ]:
f1 = (2*66.6*66.6)/(66.6+66.6)

In [ ]:
f1

In [ ]:
f2 = (2*60*100)/(60+100)
f2


In [ ]:
f1p = (2*50*50)/(50+50)
f1p

In [ ]:
P = 4/4
R = 4/4
f2p = (2*P*R)/(P+R)
f2p

In [ ]:
print('Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras vezes degustei fora de casa sabores t\u00E3o marcantes e harm\u00F4nicos! O ambiente \u00E9 familiar e iluminado. O novo espa\u00E7o ficou incr\u00EDvel. Frequento e Recomendo! \u2764\uFE0F\nParab\u00E9ns.  \uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB'.encode('utf-16', 'surrogatepass').decode('utf-16'))
print(type('Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras veze'))
# import nltk
# nltk.download('punkt')
# from textblob import TextBlob
#text = 'Conhe\u00E7o e aprecio comida \u00E1rabe desde muito pequena. Minha av\u00F3 cozinhava divinamente. \nRaras vezes degustei fora de casa sabores t\u00E3o marcantes e harm\u00F4nicos! O ambiente \u00E9 familiar e iluminado. O novo espa\u00E7o ficou incr\u00EDvel. Frequento e Recomendo! \u2764\uFE0F\nParab\u00E9ns.  \uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB\uD83D\uDC4F\uD83C\uDFFB \nFoi muito ruim demais! \n Não irei voltar mais neste lugar'.encode('utf-16', 'surrogatepass').decode('utf-16')
text = 'Sushiloko j\u00E1 foi melhor e mais barato. claro que continua bom. mas os pre\u00E7os dos combos j\u00E1 n\u00E3o est\u00E3o mais em conta'.encode('utf-16', 'surrogatepass').decode('utf-16')
comentario_blob = TextBlob(text)
comentario_blob_us = comentario_blob.translate(to='us')

for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.subjectivity)

In [ ]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.polarity)